In [23]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [24]:
mnist_base_path="/Users/zhouwencheng/Desktop/Grass/data/picture/mnist"
# 载入数据集
mnist = input_data.read_data_sets(mnist_base_path, one_hot=True)

Extracting ./data/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [26]:
# 每个批次的大小
batch_size = 100
# 计算一共有多少个批次 (550)
n_batch = mnist.train.num_examples // batch_size

# 初始化权值
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1) # 生成一个截断的正太分布
    return tf.Variable(initial)

# 初始化偏置值
def bias_variable(shape):
    initial = tf.constant(0.1,  shape = shape)  
    return tf.Variable(initial)

# 卷积层
def conv2d(x, W):
    # x 输入一个tensor of shape [batch, in_height, in_width, in_channels]
    # w filter(滤波器，卷积核) shape = [filter_height, filter_width, in_channels out_channels]
    # strides 步长
    # padding  补0不补0
    return tf.nn.conv2d(x, W, strides = [1, 1, 1, 1], padding = 'SAME')

# 池化层
def max_pool_2x2(x):
    # ksize [1, x, y, 1]
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding = 'SAME')

# 定义两个placeholder
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

# 改变x的格式专为4D向量 [batch, in_height, in_width, in_channels]
x_imge = tf.reshape(x, [-1, 28, 28, 1])

# 初始化第一个卷积层的权值和偏置
W_conv1 = weight_variable([5, 5, 1, 32]) # 5*5 的采用窗口，32个卷积核从1个平面抽取特征
b_conv1 = bias_variable([32]) # 每个卷积核一个偏置

# 把x_image和权值向量进行卷积，再加上偏置值然后用于relu激活函数
h_conv1 = tf.nn.relu(conv2d(x_imge, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1) # 进行max_pooling


W_conv2 = weight_variable([5, 5, 32, 64]) # 5*5 的采用窗口，32个卷积核从1个平面抽取特征
b_conv2 = bias_variable([64]) # 每个卷积核一个偏置

# 把x_image和权值向量进行卷积，再加上偏置值然后用于relu激活函数
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2) # 进行max_pooling


# 28 * 28 的图片第一次卷积后还是 28 *28 * 32 第一次池化 14 * 14 * 32
# 第二次卷积后 14 * 14 * 64 第二次池化后 7 * 7 * 64
# 经过上面的操作后，得到64张7*7的平面

# 初始化第一个全连接的权值 
W_fc1 = weight_variable([7*7*64, 1024])  
b_fc1 = bias_variable([1024])  # 1024个节点

# 把池化层2的输出扁平化为1维
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
# 求第一个全连接层的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# 用 keep_prob来表示神经元输出概率
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 初始化第二个全连接层
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

# 计算输出
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

# 交叉熵
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = prediction))

train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

# 结果放在一个布尔型列表中
correct_prediction = tf.equal(tf.arg_max(y, 1), tf.arg_max(prediction, 1))
# 求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict = {x:batch_xs, y:batch_ys, keep_prob:0.7})
        acc = sess.run(accuracy, feed_dict = {x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})
        print('accuracy:', acc)
print('OK')

accuracy: 0.9574


KeyboardInterrupt: 

In [ ]:
# 太慢了
- 第一次  0.9574  在21次后应该可以达到0.99 